# Load model

In [1]:
!pip3 install imgbeddings

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for onnxruntime>=1.10.0 from https://files.pythonhosted.org/packages/7a/cf/6aa8c56fd63f53c2c485921e411269c7b501a2b4e634bd02f226ab2d5d8e/onnxruntime-1.16.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 77.9 MB/s eta 0:00:00:00:0100:01
  Created wheel for imgbeddings: filename=imgbeddings-0.1.0-py3-none-any.whl size=9320 sha256=0489bf1d96624818300b8e38180a520bb4c8172599c6031f6352d8ddd8dc9e0c
  Stored in directory: /root/.cache/pip/wheels/4e/c9/9c/5f94fd783ce863540e5999f4bc2afd1501ddaea942ee6413fc
Successfully built imgbeddings


In [2]:
from imgbeddings import imgbeddings

ibed = imgbeddings()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:672: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


patch32_v1.onnx:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/clip/processing_clip.py:144: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


# Load data

In [3]:
!conda install -y gdown

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    filelock-3.13.1            |     pyhd8ed1ab_0          15 KB  conda-forge
    gdown-4.7.1                |     pyhd8ed1ab_0          19 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          34 KB

The following NEW packages will be INSTALLED:

  filelock           conda-forge/noarch::filelock-3.13.1-pyhd8ed1ab_0

In [4]:
#Load img_drop_measurement_final.csv
!gdown --id 18CW5x2GagNpQJMeKthZH449u8VEcgzya

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=18CW5x2GagNpQJMeKthZH449u8VEcgzya
To: /kaggle/working/img_drop_measurement_final.csv
100%|███████████████████████████████████████| 1.27M/1.27M [00:00<00:00, 154MB/s]


# Vector data
- Vector
- Metadata
	- Product_id (key)
	- Path
	- Category
	- Flag


In [5]:
import json
import os
from PIL import Image
import pandas as pd
import re
from tqdm.auto import tqdm
import requests
import ast
import shutil

In [6]:
#init Dataframe df: data_final.csv
df = pd.read_csv('img_drop_measurement_final.csv')
df.head()

,Unnamed: 0,item_id,name,category,link,image_link
0,0,298128,TYSSEDAL,Wardrobes,https://www.ikea.com/sa/en/p/tyssedal-wardrobe...,['https://www.ikea.com/sa/en/images/products/t...
1,1,30248537,SNIGLAR,Nursery furniture,https://www.ikea.com/sa/en/p/sniglar-cot-beech...,['https://www.ikea.com/sa/en/images/products/s...
2,2,20398688,GRÖNLID,Armchairs & chaise longues,https://www.ikea.com/sa/en/p/groenlid-cover-fo...,['https://www.ikea.com/sa/en/images/products/g...
3,3,20396260,GRÖNLID,Armchairs & chaise longues,https://www.ikea.com/sa/en/p/groenlid-cover-fo...,['https://www.ikea.com/sa/en/images/products/g...
4,4,397048,GRÖNLID,Armchairs & chaise longues,https://www.ikea.com/sa/en/p/groenlid-cover-fo...,['https://www.ikea.com/sa/en/images/products/g...


In [7]:
!mkdir imgdata

In [8]:
def get_img(url, ids):
    res = requests.get(url, stream=True)
    file_name = 'imgdata/'+str(ids)+'_'+url[url.rfind("/")+1:-4]
    if (res.status_code == 200) and (os.path.isfile(file_name)==False):
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        print('Image sucessfully Downloaded: ',file_name)
        return file_name
    else:
        print('Image Couldn\'t be retrieved')
        return None

In [9]:
def init_img_dict(df):    
    list_img_dict = []
    
    for i in tqdm(df.index):
        try: 
            url = ast.literal_eval(df['image_link'][i])[0]
        except:
            print('There\'s no image url in this product website!')
            continue
        
        #Save image into local folder
        img_path = get_img(url, i)
        #If image failed: remove this file
        try:
            img = Image.open(img_path)
        except:
            if (img_path != None):
                !rm img_path
            continue
            
        #Get several information of image
        product_id = df['item_id'][i]
        category = df['category'][i]
        embedding = ibed.to_embeddings(img).flatten().tolist()
        new_dict = {
            'vector': embedding,
            'metadata': {
                'product_id': product_id,
                'category': category,
                'image_path': img_path,
                'flag': True
            }
        }
        
        list_img_dict.append(new_dict)
    
    return list_img_dict

In [10]:
list_dict = init_img_dict(df)

with open('database_tmp.json', 'w') as fp:
    json.dump(list_dict, fp)

  0%|          | 0/2072 [00:00<?, ?it/s]

Image sucessfully Downloaded:  imgdata/0_tyssedal-wardrobe-white-mirror-glass__0626566_pe692870_s5.jpg
Image sucessfully Downloaded:  imgdata/1_sniglar-cot-beech__0637930_pe698615_s5.jpg
Image sucessfully Downloaded:  imgdata/2_groenlid-cover-for-corner-section-sporda-dark-grey__0577171_pe668641_s5.jpg
Image sucessfully Downloaded:  imgdata/3_groenlid-cover-for-corner-section-inseros-white__0577139_pe668613_s5.jpg
Image sucessfully Downloaded:  imgdata/4_groenlid-cover-for-corner-section-ljungen-light-green__0577153_pe668627_s5.jpg
Image sucessfully Downloaded:  imgdata/5_groenlid-cover-for-corner-section-ljungen-light-red__0852501_pe780119_s5.jpg
Image sucessfully Downloaded:  imgdata/6_groenlid-cover-for-corner-section-ljungen-medium-grey__0577160_pe668629_s5.jpg
Image sucessfully Downloaded:  imgdata/7_groenlid-cover-for-corner-section-sporda-natural__0577178_pe668648_s5.jpg
Image sucessfully Downloaded:  imgdata/8_norraryd-bar-stool-with-backrest-black__0728070_pe736043_s5.jpg
Imag

In [11]:
!zip -r imgdata.zip imgdata 

  adding: imgdata/ (stored 0%)
  adding: imgdata/149_eket-cabinet-pale-yellow__1219273_pe913341_s5.jpg (deflated 13%)
  adding: imgdata/1584_vimle-cover-for-footstool-with-storage-saxemara-black-blue__0949380_pe799687_s5.jpg (deflated 2%)
  adding: imgdata/106_paerup-cover-for-3-seat-sofa-with-chaise-longue-vissle-dark-green__0950145_pe800165_s5.jpg (deflated 4%)
  adding: imgdata/1294_myllra-changing-table-with-drawers-pale-pink__0812335_pe771995_s5.jpg (deflated 2%)
  adding: imgdata/1384_graval-junior-chair__0817779_pe774132_s5.jpg (deflated 4%)
  adding: imgdata/887_idasen-high-cabinet-with-drawer-and-doors-dark-green__1020158_pe831810_s5.jpg (deflated 13%)
  adding: imgdata/1479_bingsta-armchair-vissle-dark-grey-kabusa-dark-grey__0793271_pe765307_s5.jpg (deflated 0%)
  adding: imgdata/321_hemnes-tv-bench-black-brown__0625361_pe692211_s5.jpg (deflated 4%)
  adding: imgdata/1013_vimle-cover-for-3-seat-section-gunnared-beige__0531865_pe647607_s5.jpg (deflated 5%)
  adding: imgdata/17

In [12]:
# img = Image.open('imgdata/999_hattasen-bedside-table-shelf-unit-white__1155649_pe886718_s5.jpg')
# embedding = ibed.to_embeddings(img)
# print(f"{type(embedding) = }, {embedding.dtype = }, {embedding.shape = }")
# print(embedding)